# Mock Photon Simulation - Usage

This Notebook serves the purpose to show how you can use the Mock Photon propagation. Let's start with the imports

In [1]:
import sys
sys.path.append('../')

from ananke.configurations.detector import DetectorConfiguration
from ananke.services.detector import DetectorBuilderService
from olympus.event_generation.generators import GeneratorCollection, GeneratorFactory
from olympus.event_generation.medium import MediumEstimationVariant, Medium
from olympus.event_generation.photon_propagation.mock_photons import MockPhotonPropagator


/home/janik/projects/master/env/lib/python3.8/site-packages/jax/_src/api_util.py:222: SyntaxWarning: Jitted function has invalid argnames {'interactions'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has invalid argnames {invalid_argnames} "


And continue with the definition of the Detector

In [2]:
oms_per_line = 20
dist_z = 50  # m
dark_noise_rate = 16 * 1e-5  # 1/ns
side_len = 100  # m
pmts_per_module = 16
pmt_cath_area_r = 75e-3 / 2  # m
module_radius = 0.21  # m
efficiency = 0.42 # Christian S. Number

detector_configuration = DetectorConfiguration.parse_obj(
    {
        "string": {
            "module_number": 2,
            "module_distance": 50
        },
        "pmt": {
            "efficiency": efficiency,
            "noise_rate": dark_noise_rate,
            "area": pmt_cath_area_r
        },
        "module": {
            "radius": module_radius
        },
        "geometry": {
            "type": "single",
            "side_length": 100,
        },
        "seed": 31338
    }
)

detector_service = DetectorBuilderService()
det = detector_service.get(configuration=detector_configuration)



### Use Mock Photon Simulation in Pipeline

You should not be obliged to actually call the Simulation itself. Rather than that you should pass it to the generation pipeline. For the optimal pipeline, we need a generator, the hit type we want to generate and a photon propagator, where we will use the mock propagator

In [3]:
medium = Medium(MediumEstimationVariant.PONE_OPTIMISTIC)

photon_propagator = MockPhotonPropagator(
    detector=det,
    medium=medium,
    angle_resolution=18000,
)

generator_factory = GeneratorFactory(det)

cascades_generator = generator_factory.create(
    "cascade", particle_id=11, log_minimal_energy=2, log_maximal_energy=5.5, rate=0.05
)

generator_collection = GeneratorCollection(detector=det)
generator_collection.add(cascades_generator)

collection = generator_collection.generate(
    number_of_samples=1
)

Let's see what we got!

In [4]:
print('Number of Collection: ', len(collection.records))
collection.records.df.head()

Number of Collection:  1


,location_x,location_y,location_z,orientation_x,orientation_y,orientation_z,record_id,energy,length,time,type,particle_id
0,19.860758,-1.503407,-44.374696,0.257875,0.601379,0.756204,2804265273059709421,118401.349187,3000.0,0.0,cascade,11


In [5]:
print('Number of Sources: ', len(collection.sources))
collection.sources.df.head()

Number of Sources:  124


,location_x,location_y,location_z,orientation_x,orientation_y,orientation_z,record_id,time,number_of_photons,type
0,19.728525,-1.446711,-44.241127,0.257875,0.601379,0.756204,2804265273059709421,0.336900,5,cherenkov
1,19.888407,-1.305205,-44.143703,0.257875,0.601379,0.756204,2804265273059709421,1.004028,2850,cherenkov
2,19.985806,-1.309790,-43.908794,0.257875,0.601379,0.756204,2804265273059709421,1.671156,90688,cherenkov
3,20.032734,-1.115695,-43.814674,0.257875,0.601379,0.756204,2804265273059709421,2.338284,924093,cherenkov
4,20.035336,-0.913060,-43.712231,0.257875,0.601379,0.756204,2804265273059709421,3.005413,5034425,cherenkov


In [6]:
print('Number of Hits: ', len(collection.hits))
collection.hits.df.head()

Number of Hits:  28096278


,time,string_id,module_id,pmt_id,record_id
0,0.702526,0,0,0,2804265273059709421
1,0.564093,0,0,0,2804265273059709421
2,5.575400,0,0,0,2804265273059709421
3,3.630129,0,0,0,2804265273059709421
0,1.516834,0,0,0,2804265273059709421


## Use Mock Photon Propagator alone

If you want to call it yourself, you only need sources and the according events.

In [8]:
hits = photon_propagator.propagate(collection.records, collection.sources)
hits.df.head()

,time,string_id,module_id,pmt_id,record_id
0,0.373483,0,0,0,2804265273059709421
1,0.724890,0,0,0,2804265273059709421
2,0.953914,0,0,0,2804265273059709421
3,0.154064,0,0,0,2804265273059709421
0,1.958281,0,0,0,2804265273059709421
